# Bufstream Iceberg quickstart

In [1]:
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('Jupyter')
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .set("spark.sql.catalog.bufstream-quickstart", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.bufstream-quickstart.type", "rest")
        .set("spark.sql.catalog.bufstream-quickstart.uri", "http://iceberg-rest:8181")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()


25/10/21 17:48:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
%%sql 
SHOW DATABASES;

namespace
bufstream


In [3]:
%%sql
SHOW TABLES in `bufstream`

namespace,tableName,isTemporary
bufstream,invoice,False


In [25]:
%%sql
SELECT 
    line_item.product.category.name AS category,
    SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS total_revenue_dollars,
    SUM(line_item.quantity) AS total_units_sold
FROM `bufstream`.`invoice`
LATERAL VIEW EXPLODE(val.line_items) AS line_item
GROUP BY category WITH ROLLUP
ORDER BY 
    CASE WHEN category IS NULL THEN 1 ELSE 0 END,
    category


category,total_revenue_dollars,total_units_sold
Books & Stationery,586234.160000,21784
Electronics & Accessories,625379.800000,21520
Home & Garden,571721.850000,17615
Kitchen & Dining,399166.480000,10552
Personal Care,272985.060000,11794
Sports & Outdoors,509665.440000,13056
None,2965152.790000,96321


In [30]:
%%sql
SELECT 
    line_item.product.category.name AS category,
    SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS total_revenue_dollars,
    COUNT(DISTINCT val.invoice_id) AS num_invoices,
    SUM(line_item.quantity) AS total_units_sold
FROM `bufstream`.`invoice`
LATERAL VIEW EXPLODE(val.line_items) AS line_item
GROUP BY category




category,total_revenue_dollars,num_invoices,total_units_sold
Sports & Outdoors,509665.440000,5160,13056
Books & Stationery,586234.160000,4677,21784
Home & Garden,571721.850000,5255,17615
Electronics & Accessories,625379.800000,5163,21520
Personal Care,272985.060000,4115,11794
Kitchen & Dining,399166.480000,4635,10552
